# 抽象图像风格分类（SVM版本）
## 1.载入数据

In [1]:
from sklearn import svm, model_selection, metrics
from sklearn.decomposition import PCA
import numpy as np
import os

In [2]:
data_dir = "/media/ziven/My Passport/Dataset/Painter by Numbers/140x140_style/"

def load_data_PbN(PbN_dir):
        """load Pinter by Numbers dataset"""
        train_images_Abstract_Art = np.load(os.path.join(PbN_dir, 'style_train_images_Abstract_Art.npy'))
        train_images_Abstract_Expressionism = np.load(os.path.join(PbN_dir, 'style_train_images_Abstract_Expressionism.npy'))
        train_images_Art_Informel = np.load(os.path.join(PbN_dir, 'style_train_images_Art_Informel.npy'))
        train_images_Color_Field_Painting = np.load(os.path.join(PbN_dir, 'style_train_images_Color_Field_Painting.npy'))
        train_images_Lyrical_Abstraction = np.load(os.path.join(PbN_dir, 'style_train_images_Lyrical_Abstraction.npy'))
        train_images_Concretism = np.load(os.path.join(PbN_dir, 'style_train_images_Concretism.npy'))
        train_data = np.concatenate((train_images_Abstract_Art, train_images_Abstract_Expressionism,
                                     train_images_Art_Informel, train_images_Color_Field_Painting,
                                     train_images_Lyrical_Abstraction, train_images_Concretism))

        train_labels_Abstract_Art = [0 for i in range(train_images_Abstract_Art.shape[0])]
        train_labels_Abstract_Expressionism = [1 for j in range(train_images_Abstract_Expressionism.shape[0])]
        train_labels_Art_Informel = [2 for k in range(train_images_Art_Informel.shape[0])]
        train_labels_Color_Field_Painting = [3 for l in range(train_images_Color_Field_Painting.shape[0])]
        train_labels_Lyrical_Abstraction = [4 for i1 in range(train_images_Lyrical_Abstraction.shape[0])]
        train_labels_Concretism = [5 for i2 in range(train_images_Concretism.shape[0])]
        train_labels = np.concatenate((train_labels_Abstract_Art, train_labels_Abstract_Expressionism,
                          train_labels_Art_Informel, train_labels_Color_Field_Painting,
                          train_labels_Lyrical_Abstraction, train_labels_Concretism))

        test_images_Abstract_Art = np.load(os.path.join(PbN_dir, 'style_test_images_Abstract_Art.npy'))
        test_images_Abstract_Expressionism = np.load(os.path.join(PbN_dir, 'style_test_images_Abstract_Expressionism.npy'))
        test_images_Art_Informel = np.load(os.path.join(PbN_dir, 'style_test_images_Art_Informel.npy'))
        test_images_Color_Field_Painting = np.load(os.path.join(PbN_dir, 'style_test_images_Color_Field_Painting.npy'))
        test_images_Lyrical_Abstraction = np.load(os.path.join(PbN_dir, 'style_test_images_Lyrical_Abstraction.npy'))
        test_images_Concretism = np.load(os.path.join(PbN_dir, 'style_test_images_Concretism.npy'))
        val_data = np.concatenate((test_images_Abstract_Art, test_images_Abstract_Expressionism,
                                   test_images_Art_Informel, test_images_Color_Field_Painting,
                                   test_images_Lyrical_Abstraction, test_images_Concretism))

        val_labels_Abstract_Art = [0 for m in range(test_images_Abstract_Art.shape[0])]
        val_labels_Abstract_Expressionism = [1 for n in range(test_images_Abstract_Expressionism.shape[0])]
        val_labels_Art_Informel = [2 for p in range(test_images_Art_Informel.shape[0])]
        val_labels_Color_Field_Painting = [3 for q in range(test_images_Color_Field_Painting.shape[0])]
        val_labels_Lyrical_Abstraction = [4 for j1 in range(test_images_Lyrical_Abstraction.shape[0])]
        val_labels_Concretism = [5 for j2 in range(test_images_Concretism.shape[0])]
        val_labels = np.concatenate((val_labels_Abstract_Art, val_labels_Abstract_Expressionism,
                              val_labels_Art_Informel, val_labels_Color_Field_Painting,
                              val_labels_Lyrical_Abstraction, val_labels_Concretism))

        X = np.concatenate((train_data, val_data))
        X = X.reshape([X.shape[0], -1])
        X = X = PCA(n_components=3000).fit_transform(X)
        y = np.concatenate((train_labels, val_labels))
        return X, y
X, y = load_data_PbN(data_dir)
print(X.shape)
print(y.shape)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X,y, shuffle=True, random_state=2)
print(y_train[:20])
print(y_test[:20])

(5812, 3000)
(5812,)
[2 5 1 0 3 3 0 1 0 2 2 2 2 3 1 4 1 1 2 1]
[3 1 4 1 0 0 4 2 0 2 3 1 4 0 0 2 1 1 2 2]


## 2.使用SVM进行分类
* 分类策略为：one-vs-rest
* 核函数为：radial basis function

In [5]:
clf = svm.SVC(decision_function_shape='ovr', verbose=2, kernel='rbf')
clf.fit(X_train, y_train) 

[LibSVM]

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=2)

## 3.预测结果

In [6]:
predicts = clf.predict(X_test)
print(predicts)
print(y_test)
ac_score = metrics.accuracy_score(y_test, predicts)  
cl_report = metrics.classification_report(y_test, predicts)  
print(ac_score)
print(cl_report)

[3 1 3 ..., 1 1 1]
[3 1 4 ..., 3 2 1]
0.342050929112
             precision    recall  f1-score   support

          0       0.25      0.00      0.01       238
          1       0.32      0.94      0.47       431
          2       0.75      0.04      0.07       257
          3       0.50      0.31      0.38       230
          4       0.50      0.01      0.03       154
          5       0.80      0.06      0.10       143

avg / total       0.48      0.34      0.23      1453

